<h1 align ="center"> REST API Mutiple Image Samples</h1>
<hr>
   
# Chat Completions

In [3]:
import json
import os
import requests
import base64

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup deployment_name, openai_api_base, openai_api_key and openai_api_version.

In [4]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)
    
# Setting up the deployment name
deployment_name = config_details['GPT-4V_MODEL']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base = config_details['OPENAI_API_BASE']

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# Currently OPENAI API have the following versions available: 2022-12-01. All versions follow the YYYY-MM-DD date structure.
openai_api_version = config_details['OPENAI_API_VERSION']

In [5]:

#Defect Detector
reference_image_file_path = "../../common/images/DefectDetectorReference.jfif" 
with open(reference_image_file_path, 'rb') as image_file:
    encoded_reference_image = base64.b64encode(image_file.read()).decode('ascii')

test_image_file_path = "../../common/images/DefectDetectorTest.jfif"
with open(test_image_file_path, 'rb') as image_file:
    encoded_test_image = base64.b64encode(image_file.read()).decode('ascii')

sys_message = """
You're a professional defect detector.
Your job is to compare the test image with reference image, please answer the question with "No defect detected" or "Defect detected", 
also explain your decision as detail as possible."
"""

# Construct the API request URL
api_url = f"{openai_api_base}/openai/deployments/{deployment_name}/chat/completions?api-version={openai_api_version}"

# Including the api-key in HTTP headers
headers = {
    "Content-Type": "application/json",
    "api-key": openai_api_key,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        sys_message
      ]
    },
    {
      "role": "user",
      "content": [
        "Here is the reference image", # Pass the prompt
        {
          "image": encoded_reference_image  #Pass the encoded reference image
        }
      ]
    },
    {
      "role": "user",
      "content": [
        "Here is the test image", # Pass the prompt
        {
          "image": encoded_test_image  #Pass the encoded test image
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

# Send the request and handle the response
try:
    response = requests.post(api_url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error for bad HTTP status codes
    response_content = response.json()
    print(response_content['choices'][0]['message']['content'])  # Print the content of the response
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

Defect detected.

The test image shows a can lid with apparent damage. The reference image presents a pristine, undamaged can lid. In comparison, the test image reveals several areas where the top layer of the lid is peeling or has been removed, exposing the underlying layer. This damage is evident due to the visible white areas contrasting with the golden surface surrounding them. The structural integrity and aesthetic quality of the can lid in the test image is compromised, confirming the presence of a defect.
